# Utils

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

# loading and converting npy data

In [2]:
data_mel = np.load('data/npy/data_mel.npy')
labels_mel = np.load('data/npy/labels_mel.npy')

In [14]:

# Daten in Tensoren konvertieren
data_tensor = torch.from_numpy(np.array(data_mel)).float()

# import torchvision.transforms as transforms

# # Define the mean and standard deviation for normalization
# mean_mel = data_tensor.mean()
# std_mel = data_tensor.std()

# np.save('data/npy/mean_mel.npy', mean_mel)
# np.save('data/npy/std_mel.npy', std_mel)

# # Create the transform
# transform = transforms.Normalize(mean=mean_mel, std=std_mel)

# # Reshape the data_tensor to have the shape (batch_size, channels, height, width)
# data_tensor_for_normalisazion = data_tensor.view(-1, 1, 1, data_tensor.shape[1])
# # Apply the transform to the data_tensor
# normalized_data = transform(data_tensor_for_normalisazion)

labels_tensor = torch.from_numpy(np.array(labels_mel))

# Teilen der Daten in Trainings- und Testdatensätze
X_train, X_test, y_train, y_test = train_test_split(data_tensor, labels_tensor, test_size=0.2)
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)


In [15]:
# save the data
torch.save(X_train, 'data/pt/X_train.pt')
torch.save(X_test, 'data/pt/X_test.pt')
torch.save(y_train, 'data/pt/y_train.pt')
torch.save(y_test, 'data/pt/y_test.pt')

# Loading pt data

In [3]:
X_train = torch.load('data/pt/X_train.pt')
X_test = torch.load('data/pt/X_test.pt')
y_train = torch.load('data/pt/y_train.pt')
y_test = torch.load('data/pt/y_test.pt')

# Defining Model

## with class

In [11]:
# defining knn model
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)

        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 1)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    def forward(self, x):
        # add a dimension for the channel
        x = x.unsqueeze(1)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = F.relu(self.conv4(x))
        x = self.pool(x)

        x = x.view(-1, 256 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x


In [15]:
# Modellinstanz
model = CNNModel().to(device)
print(model)

CNNModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=1, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


## loading model

In [14]:
model = torch.load('data/models/KNNmodelV1.pt')

# Defining optimizer

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.MSELoss()

# add learning rate scheduler
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

# Training model

In [17]:
model.train()
for epoch in range(50):
    optimizer.zero_grad()
    output = model(X_train)
    output = output.squeeze()  # Remove dimensions of size 1
    loss = criterion(output, y_train.float())
    loss.backward()
    optimizer.step()
    # scheduler.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}, Prediction: {output[0]}, Label: {y_train[0]}')
    if loss.item() < 1-0.91:
        print('break')
        break

RuntimeError: Given groups=1, weight of size [32, 1, 3, 3], expected input[1, 216, 1, 22144] to have 1 channels, but got 216 channels instead

## V2 of training model

In [18]:
from torch.utils.data import DataLoader

# Create a DataLoader to handle batching of the training data
batch_size = 32
train_data = torch.utils.data.TensorDataset(X_train, y_train.float())
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

for epoch in range(100):
    for batch_X, batch_y in train_loader:
        model.train()
        optimizer.zero_grad()
        output = model(batch_X)
        output = output.squeeze()  # Remove dimensions of size 1
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()} \t Prediction: {output[0]}, {batch_y[0]}')
    if loss.item() < 1-0.91:
        print('break')
        break

RuntimeError: Given groups=1, weight of size [32, 1, 3, 3], expected input[1, 32, 1, 22144] to have 1 channels, but got 32 channels instead

# Evaluate model

In [11]:
# Set the model to evaluation mode
model.eval()

# Make predictions on the test data
for i in range(len(X_test)):
    with torch.no_grad():
        output = model(X_test[i])
        output = output.squeeze()
        est_loss = criterion(output, y_test[i].float())
        print(f"pred: {output.item()}, True Label: {y_test[i]}")

pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 1.0065126419067383, True Label: 1
pred: 0.27173441648483276, True Label: 0
pred: 1.0110076665878296, True Label: 1
pred: 0.27173441648483276, True Label: 1
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 1
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 1
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 1
pred: 0.27173441648483276, True Label: 0
pred: 1.0109965801239014, True Label: 1
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.99165016412

# Safe model

In [12]:
torch.save(model, 'data/models/KNNmodelV2-2.pt')